In [2]:
from IPython.core.display import *
import numpy as np
import csv
import random
import math

In [3]:
def log2(x):
    return math.log10(x) / math.log10(2)

In [4]:
def remove(items, item):
    return [i for i in items if i != item]

In [5]:
def count(data, value):
    cnt = 0
    for d in data:
        if (d['class'] == value):
            cnt += 1
    return cnt

In [6]:
def IsDataHomogeneous(data):
    tmp = data[0]['class']
    for d in data:
        if d['class'] != tmp:
            return False
    return True

In [7]:
def get_subdata(data, attr):
    values = {'cap-shape':['b','c','x','f','k','s'], 
              'cap-surface':['f','g','y','s'], 
              'cap-color':['n','b','c','g','r','p','u','e','w','y'], 
              'bruises':['t','f'], 
              'odor':['a','l','c','y','f','m','n','p','s'],
              'gill-attachment':['a','d','f','n'],
              'gill-spacing':['c','w','d'],
              'gill-size':['b','n'],
              'gill-color':['k','n','b','h','g','r','o','p','u','e','w','y'],
              'stalk-shape':['e','t'],
              'stalk-root':['b','c','u','e','z','r','?'],
              'stalk-surface-above-ring':['f','y','k','s'],
              'stalk-surface-below-ring':['f','y','k','s'],
              'stalk-color-above-ring':['n','b','c','g','o','p','e','w','y'],
              'stalk-color-below-ring':['n','b','c','g','o','p','e','w','y'],
              'veil-type':['p','u'],
              'veil-color':['n','o','w','y'],
              'ring-number':['n','o','t'],
              'ring-type':['c','e','f','l','n','p','s','z'],
              'spore-print-color':['k','n','b','h','r','o','u','w','y'],
              'population':['a','c','n','s','v','y'],
              'habitat':['g','l','m','p','u','w','d']
             }
    return [(v, [d for d in data if d[attr] == v]) for v in values[attr]]

In [8]:
def entropy(data):
    values = remove([count(data, v) for v in ['e', 'p']], 0)
    s = float(sum(values))
    return sum([-(v/s)*log2(v/s) for v in values])

In [9]:
def gain(data, attr):
    n = float(len(data))
    r = 0
    for (v, subdata) in get_subdata(data, attr):
        r += (len(subdata)/n)*entropy(subdata)
    return entropy(data) - r

In [10]:
def majority_label(data):
    set1 = [d for d in data if d['class'] == 'e']
    set2 = [d for d in data if d['class'] == 'p']
    if len(set1) > len(set2):
        return 'e'
    elif len(set1) < len(set2):
        return 'p'
    else:
        random.choice['e', 'p']

In [11]:
def pick_best_attribute(data, attributes):
    best_attr = attributes[0]
    best_attr_gain = gain(data, best_attr)
    for attr in attributes:
        attr_gain = gain(data, attr)
        if (attr_gain > best_attr_gain):
            best_attr = attr
            best_attr_gain = attr_gain
    return best_attr

In [12]:
data = []
attributes = ['cap-shape', 'cap-surface', 'cap-color', 'bruises', 
              'odor', 'gill-attachment', 'gill-spacing', 'gill-size',
              'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
              'stalk-surface-below-ring', 'stalk-color-above-ring', 
              'stalk-color-below-ring', 'veil-type', 'veil-color',
              'ring-number', 'ring-type', 'spore-print-color',
              'population', 'habitat']
with open('agaricus-lepiota.data', 'r') as csvfile:
    datareader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in datareader:
        inputs = [v for v in row]
        d = {}
        d['class'] = inputs[0]
        index = 1
        for a in attributes:
            d[a] = inputs[index]
            index += 1
        data.append(d)
        
random.shuffle(data)

In [13]:
n = len(data)/2
data1 = data[0:n]
data2 = data[n:]

In [14]:
class Node:
    def __init__(self, attribute, children=None):
        self.attribute = attribute
        if children != None:
            self.children = children
        else:
            self.children = {}
    def add(self, attr, child):
        self.children[attr] = child
        return self
    def classify(self, test_data):
        child = self.children[test_data[self.attribute]]
        if isinstance(child, Node):
            return child.classify(test_data)
        else:
            return child
    def __repr__(self):
        return 'Node(%r, %r)' % (self.attribute, self.children)

In [15]:
def id3(data, attributes, default=None):
    if len(data) == 0:
        return default
    elif IsDataHomogeneous(data):
        return data[0]['class']
    elif len(attributes) == 0:
        return majority_label(data)
    else:
        best_attr = pick_best_attribute(data, attributes)
        node = Node(best_attr)
        for (value, subdata) in get_subdata(data, best_attr):
            child = id3(subdata, remove(attributes, best_attr), majority_label(data))
            node.add(value, child)
        return node

In [16]:
def classify(tree, test_data):
    result = []
    for d in test_data:
        result.append(tree.classify(d))
    return result

In [17]:
def evaluate(test_data, classified_data):
    errors = 0
    n = len(test_data)
    for i in range(n):
        if test_data[i]['class'] != classified_data[i]:
            errors += 1
    error_rate = float(errors)/n
    return error_rate

In [18]:
tree1 = id3(data1, attributes)

In [19]:
print tree1

Node('odor', {'a': 'e', 'c': 'p', 'f': 'p', 'm': 'p', 'l': 'e', 'n': Node('spore-print-color', {'b': 'e', 'h': 'e', 'k': 'e', 'o': 'e', 'n': 'e', 'r': 'p', 'u': 'e', 'w': Node('habitat', {'d': Node('gill-size', {'b': 'e', 'n': 'p'}), 'g': 'e', 'm': 'e', 'l': Node('cap-color', {'c': 'e', 'b': 'e', 'e': 'e', 'g': 'e', 'n': 'e', 'p': 'e', 'r': 'e', 'u': 'e', 'w': 'p', 'y': 'p'}), 'p': 'e', 'u': 'e', 'w': 'e'}), 'y': 'e'}), 'p': 'p', 's': 'p', 'y': 'p'})


In [20]:
classified_data1 = classify(tree1, data2)
for i in range(len(data2)):
    print classified_data1[i], data2[i]['class']
evaluate(data2, classified_data1)

p p
e e
p p
e e
p p
e e
e e
e e
p p
p p
p p
p p
e e
p p
p p
e e
e e
e e
e e
p p
p p
p p
e e
e e
e e
e e
e e
e e
p p
e e
e e
p p
e e
p p
e e
p p
e e
p p
e e
p p
e e
e e
e e
e e
e e
e e
e e
p p
e e
p p
e e
e e
e e
e e
p p
e e
e e
p p
p p
e e
e e
e e
e e
p p
e e
p p
p p
e e
p p
e e
p p
p p
e e
p p
p p
e e
e e
p p
e e
p p
p p
e e
e e
p p
p p
e e
e e
e e
p p
p p
p p
e e
p p
p p
p p
e e
e e
e e
p p
e e
e e
e e
e e
e e
p p
p p
p p
e e
e e
p p
p p
e e
p p
e e
e e
e e
p p
e e
e e
e e
p p
e e
e e
p p
e e
e e
p p
e e
e e
p p
e e
p p
e e
p p
e e
e e
e e
e e
e e
p p
p p
e e
p p
e e
p p
e e
p p
e e
p p
p p
p p
p p
p p
p p
p p
p p
p p
e e
e e
e e
p p
e e
p p
p p
e e
e e
p p
p p
e e
e e
e e
p p
p p
e e
p p
e e
e e
p p
e e
e e
p p
p p
p p
e e
e e
e e
e e
e e
e e
e e
e e
e e
p p
e e
e e
e e
p p
p p
p p
e e
p p
p p
e e
e e
e e
p p
e e
e e
e e
p p
e e
e e
e e
e e
e e
e e
e e
e e
e e
p p
p p
p p
e e
p p
e e
e e
e e
e e
p p
p p
e e
p p
e e
e e
p p
p p
p p
e e
e e
p p
p p
e e
e e
p p
e e
e e
p p
p p
e e
e e


0.0

In [21]:
tree2 = id3(data2, attributes)

In [22]:
print tree2

Node('odor', {'a': 'e', 'c': 'p', 'f': 'p', 'm': 'p', 'l': 'e', 'n': Node('spore-print-color', {'b': 'e', 'h': 'e', 'k': 'e', 'o': 'e', 'n': 'e', 'r': 'p', 'u': 'e', 'w': Node('cap-color', {'c': 'e', 'b': 'e', 'e': 'e', 'g': 'e', 'n': Node('stalk-surface-above-ring', {'y': 'e', 'k': 'p', 's': 'e', 'f': 'e'}), 'p': 'e', 'r': 'e', 'u': 'e', 'w': Node('cap-surface', {'y': 'p', 's': 'e', 'g': 'p', 'f': 'e'}), 'y': 'p'}), 'y': 'e'}), 'p': 'p', 's': 'p', 'y': 'p'})


In [23]:
classified_data2 = classify(tree2, data1)
for i in range(len(data1)):
    print classified_data2[i], data1[i]['class']
evaluate(data1, classified_data2)

e e
p p
p p
e e
p p
e e
e e
p p
p p
e e
p p
p p
e e
p p
p p
e e
p p
p p
e e
p p
p p
p p
e e
p p
e e
p p
e e
p p
p p
e e
e e
e e
e e
e e
e e
e e
e e
p p
e e
e e
e e
p p
e e
p p
p p
p p
e e
e e
p p
p p
e e
e e
p p
e e
p p
e e
e e
e e
e e
e e
e e
e e
e e
p p
e e
e e
p p
e e
e e
e e
e e
e e
p p
e e
p p
p p
e e
e e
p p
e e
e e
p p
e e
p p
e e
p p
p p
p p
p p
e e
e e
p p
e e
p p
p p
e e
e e
p p
p p
e e
e e
p p
p p
e e
p p
p p
e e
e e
p p
p p
e e
p p
p p
p p
p p
p p
p p
e e
p p
p p
p p
e e
e e
e e
e e
p p
e e
e e
p p
e e
e e
e e
p p
e e
p p
p p
e e
p p
p p
e e
p p
e e
e e
p p
e e
p p
p p
p p
p p
p p
e e
p p
p p
p p
p p
e e
e e
e e
e e
e e
p p
e e
e e
p p
p p
e e
e e
p p
e e
e e
p p
p p
p p
e e
p p
e e
p p
p p
e e
e e
p p
e e
p p
p p
p p
p p
e e
p p
p p
e e
p p
e e
e e
e e
e e
p p
e e
p p
e e
p p
e e
p p
p p
p p
e e
e e
p p
p p
e e
p p
e e
e e
p p
e e
p p
e e
e e
p p
p p
p p
e e
p p
p p
p p
p p
p p
e e
e e
p p
e e
p p
p p
e e
p p
p p
p p
p p
p p
p p
p p
e e
p p
e e
p p
p p
e e
p p
e e
e e
e e


0.0